<td>   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a></td>


<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/model_experiments/model_predictions_to_project.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/develop/examples/model_experiments/model_predictions_to_project.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Import Model Run Predictions to a Project
Throughout the process of training your machine learning (ML) model, you may want to export your model-run predictions and import them to your new project. In this notebook, we will demonstrate the process on how to get those predictions moved over.

In [ ]:
%pip install -q "labelbox[data]"

In [ ]:
import labelbox as lb
import labelbox.types as lb_types
from labelbox.schema.conflict_resolution_strategy import (
    ConflictResolutionStrategy,)
import uuid

## API Key and Client
See the developer guide for [creating an API key](https://docs.labelbox.com/reference/create-api-key).

In [ ]:
# Add your API key
API_KEY = ""
# To get your API key go to: Workspace settings -> API -> Create API Key
client = lb.Client(api_key=API_KEY)

### Creating Model Experiment

In order to interact with Model Run predictions, you must create a Model Experiment with a Model Run and then add predictions. The steps below go over this process. See [Model](https://docs.labelbox.com/reference/model) from our developer guides for more information.

To create a Model Experiment you will need to create an ontology. See [Ontology](https://docs.labelbox.com/reference/ontology) for more information

#### Ontology

In this example we are making a simple ontology with a classification feature. The classification feature has two options: option 1 and option 2.

In [ ]:
classification_features = [
    lb.Classification(
        class_type=lb.Classification.Type.CHECKLIST,
        name="Demo Feature",
        options=[lb.Option(value="option 1"),
                 lb.Option(value="option 2")],
    )
]

ontology_builder = lb.OntologyBuilder(tools=[],
                                      classifications=classification_features)

ontology = client.create_ontology("Demo Ontology",
                                  ontology_builder.asdict(),
                                  media_type=lb.MediaType.Image)

#### Model Experiment

In [ ]:
model = client.create_model(name=f"Model Experiment Demo {str(uuid.uuid4())}",
                            ontology_id=ontology.uid)

### Creating a Model Run from Model Experiment

On this step we will need to create a dataset to attach data rows to our model run. See [Dataset](https://docs.labelbox.com/reference/dataset) for more information.

#### Dataset and Data Rows

In [ ]:
# send a sample image as data row for a dataset
global_key = "2560px-Kitano_Street_Kobe01s5s4110" + str(uuid.uuid4())

test_img_url = {
    "row_data":
        "https://storage.googleapis.com/labelbox-datasets/image_sample_data/2560px-Kitano_Street_Kobe01s5s4110.jpeg",
    "global_key":
        global_key,
}

dataset = client.create_dataset(name="foundry-demo-dataset")
task = dataset.create_data_rows([test_img_url])
task.wait_till_done()

print(f"Errors: {task.errors}")
print(f"Failed data rows: {task.failed_data_rows}")

if task.errors:
    for error in task.errors:
        if ("Duplicate global key" in error["message"] and
                dataset.row_count == 0):
            # If the global key already  exists in the workspace the dataset will be created empty, so we can delete it.
            print(f"Deleting empty dataset: {dataset}")
            dataset.delete()

#### Create Model Run and Attach Data Rows

In [ ]:
model_run_name = f"Model Run Demo {str(uuid.uuid4())}"

model_run = model.create_model_run(name=model_run_name)

In [ ]:
model_run.upsert_data_rows(global_keys=[global_key])

#### Add Predictions
In the below code snippet we are adding a sample predictions and attaching them to our data row inside our model run.

In [ ]:
checklist_prediction = lb_types.ClassificationAnnotation(
    name="Demo Feature",
    value=lb_types.Checklist(
        answer=[lb_types.ClassificationAnswer(name="option 1", confidence=0.5)
               ]),
)

# Create prediction label
label_prediction = [
    lb_types.Label(
        data=lb_types.ImageData(global_key=global_key),
        annotations=[checklist_prediction],
    )
]

# Upload the prediction label to the Model Run
upload_job_prediction = model_run.add_predictions(
    name="prediction_upload_job" + str(uuid.uuid4()),
    predictions=label_prediction,
)

# Errors will appear for prediction uploads that failed.
print("Errors:", upload_job_prediction.errors)
print("Status of uploads: ", upload_job_prediction.statuses)

### Setup Project and Add Predictions
In the steps below we will be creating our target project and setting up the project with the ontology we used with our model run. See [Project](https://docs.labelbox.com/reference/dataset) for more information.

#### Project 

In [ ]:
# Create a new project
project = client.create_project(name="Model Run Import Demo Project",
                                media_type=lb.MediaType.Image)

In [ ]:
# Setup Ontology
project.setup_editor(ontology)

#### Ontology Mapping
To send prediction to your annotate project you will need to provide a ontology mapping python dictionary item. This matches ontology feature id to another. You would use this if your ontology was different from your model run to your project. In our case, since we are using the same ontology, you would just need to map the same feature id to each other.

In [ ]:
# Get ontology dictionary to obtain featureSchemaIds
ontology_normalized = ontology.normalized

PREDICTIONS_ONTOLOGY_MAPPING = {
    ontology_normalized["classifications"][0]["featureSchemaId"]:
        ontology_normalized["classifications"][0]
        ["featureSchemaId"],  # Classification featureSchemaID
    ontology_normalized["classifications"][0]["options"][0]["featureSchemaId"]:
        ontology_normalized["classifications"][0]["options"][0]
        ["featureSchemaId"],  # Different Classification Answer featureSchemaIDs
    ontology_normalized["classifications"][0]["options"][1]["featureSchemaId"]:
        ontology_normalized["classifications"][0]["options"][1]
        ["featureSchemaId"],
}

#### Send Model Predictions from Model Run to Annotate
To send our predictions to our project we will be using the `send_to_annotate_from_model` method from our project. See [Foundry apps](https://docs.labelbox.com/reference/foundry-2#send-foundry-generated-annotations-from-catalog-to-annotate) for more information.
##### Parameters

When you send predicted data rows to annotate from a model run, you may choose to include or exclude certain parameters, at a minimum a predictions_ontology_mapping will need to be provided:

* `predictions_ontology_mapping`
    - A dictionary containing the mapping of the model's ontology feature schema ids to the project's ontology feature schema ids
* `exclude_data_rows_in_project`
    - Excludes data rows that are already in the project. 
* `override_existing_annotations_rule` 
    - The strategy defining how to handle conflicts in classifications between the data rows that already exist in the project and incoming predictions from the source model run or annotations from the source project. 
        * Defaults to ConflictResolutionStrategy.KeepExisting
        * Options include:
            * ConflictResolutionStrategy.KeepExisting
            * ConflictResolutionStrategy.OverrideWithPredictions
            * ConflictResolutionStrategy.OverrideWithAnnotations
* `param batch_priority`
    - The priority of the batch.


#### Import Predictions as pre-labels

In [ ]:
send_to_annotations_params = {
    "predictions_ontology_mapping":
        PREDICTIONS_ONTOLOGY_MAPPING,
    "exclude_data_rows_in_project":
        False,
    "override_existing_annotations_rule":
        ConflictResolutionStrategy.OverrideWithPredictions,
    "batch_priority":
        5,
}

# Send the predictions as pre-labels
queue_id = [
    queue.uid
    for queue in project.task_queues()
    if queue.queue_type == "INITIAL_LABELING_QUEUE"
][0]

task = model_run.send_to_annotate_from_model(
    destination_project_id=project.uid,
    task_queue_id=
    queue_id,  # ID of workflow task, set ID to None if you want to convert pre-labels to ground truths .
    batch_name="Prediction Import Demo Batch",
    data_rows=lb.GlobalKeys(
        [global_key]  # Provide a list of global keys from foundry app task
    ),
    params=send_to_annotations_params,
)

task.wait_till_done()

print(f"Errors: {task.errors}")

## Cleanup

In [ ]:
# project.delete()
# dataset.delete()
# model_run.delete()